In [1]:
import keyring

keyring.set_password('tiingo', 'charmingcity','134661f9fc9106d35b3272ae79f125cf29252685')

In [2]:
from tiingo import TiingoClient
import pandas as pd 
import keyring 

In [3]:
api_key = keyring.get_password('tiingo', 'charmingcity')
config = {}
config['session'] = True
config['api_key'] = api_key
client = TiingoClient(config) #접속환경

In [4]:
tickers = client.list_stock_tickers() #list_stcok_tickers() 메서드를 통해서 티커 정보를 가져올 수 있다.
tickers_df = pd.DataFrame.from_records(tickers)

In [5]:
ticker_metadata = client.get_ticker_metadata("AAPL") #get_ticker_metadata() 메서드 내에 티커 입력
print(ticker_metadata)


{'ticker': 'AAPL', 'name': 'Apple Inc', 'description': "Apple Inc. (Apple) designs, manufactures and markets mobile communication and media devices, personal computers, and portable digital music players, and a variety of related software, services, peripherals, networking solutions, and third-party digital content and applications. The Company's products and services include iPhone, iPad, Mac, iPod, Apple TV, a portfolio of consumer and professional software applications, the iOS and OS X operating systems, iCloud, and a variety of accessory, service and support offerings. The Company also delivers digital content and applications through the iTunes Store, App StoreSM, iBookstoreSM, and Mac App Store. The Company distributes its products worldwide through its retail stores, online stores, and direct sales force, as well as through third-party cellular network carriers, wholesalers, retailers, and value-added resellers. In February 2012, the Company acquired app-search engine Chomp.", 

In [6]:
historical_prices = client.get_dataframe("AAPL",                   #get_dataframe() 메서드 내에 티커를 입력하면 close(종가), high(고가), low(저가), open(시가), volumne(거래량), 수정주가, divCash(현금배당), splitFactor(주식분할 조정계수) 
                                         startDate= '2017-01-01',
                                         frequency='daily')
historical_prices.head()

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
date,,,,,,,,,,,,
2017-01-03 00:00:00+00:00,116.15,116.3300,114.76,115.80,28781865,27.130159,27.172204,26.805485,27.048407,115127460,0.0,1.0
2017-01-04 00:00:00+00:00,116.02,116.5100,115.75,115.85,21118116,27.099794,27.214248,27.036728,27.060086,84472464,0.0,1.0
2017-01-05 00:00:00+00:00,116.61,116.8642,115.81,115.92,22193587,27.237606,27.296981,27.050743,27.076436,88774348,0.0,1.0
2017-01-06 00:00:00+00:00,117.91,118.1600,116.47,116.78,31751900,27.541258,27.599653,27.204905,27.277314,127007600,0.0,1.0
2017-01-09 00:00:00+00:00,118.99,119.4300,117.94,117.95,33561948,27.793523,27.896297,27.548265,27.550601,134247792,0.0,1.0


In [7]:
fundamentals_daily = client.get_fundamentals_daily('AAPL')         #get_fundamentals_daily() 메서드 내에 티커를 입력하면 시가총액, 기업가치, PER, PBR, PEG 정보가 JSON 형태로 받아지며, from_record() 메서드를 통해 데이터프레임 형태로 변경해 준다. 
fundamentals_daily_df = pd.DataFrame.from_records(fundamentals_daily)

fundamentals_daily_df.head()

,date,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y
0,2020-04-22T00:00:00.000Z,1.208070e+12,1.277403e+12,21.114568,15.404144,5.931274
1,2020-04-23T00:00:00.000Z,1.203388e+12,1.272721e+12,21.032741,15.344447,5.908288
2,2020-04-24T00:00:00.000Z,1.238130e+12,1.307463e+12,21.639947,15.787435,6.078858
3,2020-04-27T00:00:00.000Z,1.239005e+12,1.308338e+12,21.655242,15.798593,6.083154
4,2020-04-28T00:00:00.000Z,1.218921e+12,1.288254e+12,21.304225,15.542508,5.984550


In [8]:
fundametnals_stmnts = client.get_fundamentals_statements(        #get_fundamentlas_statements() 메서드 내에 티커를 입력하면 재무제표의 세부 항목을 받을 수 있다. fmt는 포맷 형태를 의미한다. JSON보다 복잡도가 낮은 CSV로 받는다.    
    'AAPL', startDate='2019-01-01', asReported=True, fmt='csv')

In [10]:
df_fs = pd.DataFrame([x.split(',') for x in fundametnals_stmnts.split('\n')]) #텍스트 형태로 데이터가 들어오므로, 클랜징을 통해 데이터프레임 형태로 변경한다. 
df_fs.columns = df_fs.iloc[0]    #첫 번째 행을 열 이름으로 지정한 후, 해당 행은 삭제한다. 
df_fs = df_fs[1:]
df_fs.set_index('date', drop=True, inplace=True) #'date'열을 인덱스로 지정한다. 
df_fs = df_fs[df_fs.index != ''] # 'date'가 비어 있는 부분이 있으므로 이를 제거한다.

df_fs

,year,quarter,statementType,dataCode,value
date,,,,,
2023-02-03,2023,1,cashFlow,depamor,2916000000.0
2023-02-03,2023,1,balanceSheet,ppeq,42951000000.0
2023-02-03,2023,1,balanceSheet,intangibles,0.0
2023-02-03,2023,1,incomeStatement,opinc,36016000000.0
2023-02-03,2023,1,incomeStatement,shareswa,15892723000.0
...,...,...,...,...,...
2020-10-30,2020,0,incomeStatement,rnd,18752000000.0
2020-10-30,2020,0,incomeStatement,ebt,67091000000.0
2020-10-30,2020,0,balanceSheet,liabilitiesNonCurrent,153157000000.0
